In [ ]:
from tqdm import tqdm_notebook as tqdm
import json
from collections import defaultdict
import csv
import os
import random
import pandas as pd
import pickle
import numpy as np
import re

In [ ]:
data_dict = {}
tot = 0
for file in tqdm(os.listdir('./extracted_data/')):
    with open('./extracted_data/'+file,'r') as fil:
        data = json.load(fil)
    tot += len(data)
    for key in data:
        data_dict[key] = data[key]
        data_dict[key]['site_id'] = "-".join((key.split("-"))[0:2])
        
reviews = pd.DataFrame.from_dict(data_dict, orient='index')
reviews.reset_index(level=0, inplace=True)

In [ ]:
with open('all_reviews.pkl','wb+') as f:
    pickle.dump(reviews,f)

In [ ]:
with open('all_reviews.pkl','rb+') as f:
    reviews = pickle.load(f)

In [ ]:
keywords = {}
keywords['family']= ['family','kids','sons']
keywords['couples'] = ['girlfriend','boyfriend','husband','wife','couple']
keywords['handicap'] = ['wheelchair','handicap','elevator','disabled','stairs']
keywords['crowd'] = ['crowd','jammed','people','packed','queue']
keywords['claustrophobic'] = ['claustrophob']
keywords['time_of_day'] = ['day','night','afternoon','morning','evening']
keywords['price'] = ['expensive','cheap','pricey','price','cost','$','dollars','overpriced']
keywords['food'] = ['delicious','food','meat','din','lunch','veg']
keywords['age'] = ['young','old','year','teen','grandparents']

In [ ]:
categories = {}
for key in keywords.keys():
    categories[key] = []

for i,row in tqdm(reviews.iterrows(),total=len(reviews)):
    for cat in categories.keys():
        for word in keywords[cat]:
            if word in str(row['text']):
                categories[cat].append(row)
                break

In [ ]:
sampled_reviews = {}
uniq = []
for key in categories:
    site_rev = defaultdict(lambda:[])
    sampled_reviews[key] = []
    for row in categories[key]:
        site_rev[row['site_id']].append(row)
    
    sites = random.sample(list(site_rev.keys()),205)
    for site in sites:
        while True:
            samp = random.sample(site_rev[site],1)[0]
            if samp['index'] not in uniq:
                break
        sampled_reviews[key].append(samp)
        uniq.append(samp['index'])

In [ ]:
idli = []
for key in sampled_reviews:
    for rev in sampled_reviews[key]:
        idli.append(rev['index'])
print(len(idli))
print(len(set(idli)))

idlis = []
for key in categories:
    for rev in categories[key]:
        idlis.append(rev['index'])

idlis = list(set(idlis))

sampled_reviews['other'] = []
for i in tqdm(range(155)):
    while True:
        rno = random.randint(0,len(reviews)-1)
        row = reviews.iloc[rno]
        if row['index'] not in idlis:
            break
    idlis.append(row['index'])
    sampled_reviews['other'].append(row)

with open('sampled.pkl','wb+') as f:
    pickle.dump(sampled_reviews,f)

def clean_text(text):
    text = re.sub('\\\\n','',text)
    text = re.sub('\\\\u2019','\'',text) 
    text = re.sub(u' [a-zA-Z0-9]*\\\\u[a-zA-Z0-9]* ','',text)
    text = re.sub('\(','',text)
    text = re.sub('\)','',text)
    text = re.sub('<','< ',text)
    text = re.sub('>',' >',text)
    text = re.sub('\[','\[ ',text)
    text = re.sub('\]',' \]',text)
    return text

review_list = []
for cat in sampled_reviews:
    for row in sampled_reviews[cat]:
        temp = {}
        text = clean_text(row['text'])
        temp['text'] = re.sub('\\\\','',text)
        temp['meta'] = {}
        temp['meta']['review_id'] = row['index']
        temp['meta']['category'] = cat
        review_list.append(json.dumps(temp))
random.shuffle(review_list)

with open('final_annotations.jsonl','w') as f:
    for lis in review_list:
        f.write(str(lis)+'\n')